

| column name    | meaning                 |                                                                                                                         
| :-------------- | :----------------------- |                                                        
| family_fot     | 英文科名                |  
| family_fot_zh  | 中文科名                |     
| family         | Family (APG IV & PPG I) |     
| family_zh      | 科名 (APG IV & PPG I)   |      
| fullname       | 學名                    |       
| zh_name        | 中文名                  |       
| prev_category  | 2008-2010 Red List      |     
| prev_criteria  | 評估標準                |       
| prev_evaluator | 評估者                  |      
| category       | 2017 Red List           |    
| criteria       | 2017評估標準            |    
| evaluator      | 2017評估者              |      
| evaluation     | 2017評估說明書          |   
| images         | 2017圖片                |    
| endemic        | 特有性                  |      
| native         | 原生性                  |      
| others         | 其他                    |      
| note           | 備註                    |     

In [1]:
import pandas as pd
import psycopg2 as pg
import genlist_api
import re
import csv
g = genlist_api.Genlist()
# postgresql database name
DSN = 'dbname=nvdimp user=psilotum'
conn = pg.connect(DSN)

## Create data table
twredlist2017

In [ ]:
#id|family_fot|family_fot_zh|family|family_zh|fullname|
#zh_name|prev_category|prev_criteria|prev_evaluator|
#category|criteria|evaluator|evaluation|images|endemic|native|
#others|note|update

CREATE_TABLE = '''
CREATE TABLE nomenclature.twredlist2017_backup AS (SELECT * FROM nomenclature.twredlist2017);
DROP TABLE IF EXISTS nomenclature.twredlist2017;                                                                                                                                   
CREATE TABLE nomenclature.twredlist2017 (                                                                                                                                          
    id serial primary key,                                                                                                                                            
    plant_type integer,                                                                                                                                               
    family_fot varchar,                                                                                                                                               
    family_fot_zh varchar,                                                                                                                                            
    family varchar,                                                                                                                                                   
    family_zh varchar,                                                                                                                                                
    genus varchar,                                                                                                                                                    
    genus_zh varchar,                                                                                                                                                 
    name varchar,                                                                                                                                                     
    fullname varchar,                                                                                                                                                 
    zh_name varchar,                                                                                                                                                  
    authors varchar,                                                                                                                                                  
    prev_category varchar,                                                                                                                                            
    prev_criteria varchar,                                                                                                                                            
    prev_evaluator varchar,                                                                                                                                           
    category varchar,                                                                                                                                                 
    criteria varchar,                                                                                                                                                 
    evaluator varchar,                                                                                                                                                
    evaluation varchar,                                                                                                                                               
    images varchar,                                                                                                                                                   
    endemic varchar,                                                                                                                                                  
    native varchar,                                                                                                                                                   
    others varchar,                                                                                                                                                   
    note varchar,
    update varchar,
    tropicos_namecode varchar,
    tropicos_fullname varchar,
    ipni_id varchar,
    ipni_fullname varchar,
    ipni_name varchar
);
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(CREATE_TABLE)
    conn.commit()

In [ ]:
# 匯入 csv 檔
# 匯入前要把斷行檢查一下
# Limnocharis flava
# Sorghum bicolor subsp. arundinaceum
# 

redlistRaw = []
with open('../../workspace/redlist171211.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
r = redlistRaw

#rlistRaw = pd.read_csv('../../workspace/redlist171204.csv', delimiter='|', na_values='')

In [ ]:
for row in range(len(r)):
    # escape quote (for postgresql)
    if "'" in r[row][5]:
        r[row][5] = r[row][5].replace("'", "''")
    eachRow = "'" + "','".join(r[row]) + "'"
    # replace full-width braces
    eachRow = eachRow.replace('（', ' (')
    eachRow = eachRow.replace(' ）', ') ')
    # replace 雜交的 "x" (alphabet) to × (cross sign)
    eachRow = re.sub(' x ', ' × ', eachRow)
    # replace spaces greater than 1 
    eachRow = re.sub(" {1,}",' ', eachRow)
    # replace trailing spaces 
    eachRow = re.sub(" {1,}'","'", eachRow)
    # replace non-break spaces
    eachRow = eachRow.replace('\xa0',' ')
    # replace ssp. to subsp.
    eachRow = re.sub(" ssp. ", " subsp. ", eachRow)
    
    INSERT_REDLIST = '''
    INSERT INTO nomenclature.twredlist2017 
    (id,family_fot,family_fot_zh,family,family_zh,fullname,                                                                                                                   
    zh_name,prev_category,prev_criteria,prev_evaluator,                                                                                                                   
    category,criteria,evaluator,evaluation,images,endemic,                                                                                                                
    native,others,note,update) VALUES (%s)
    ''' % (eachRow)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(INSERT_REDLIST)
        conn.commit()

In [ ]:
# 更新 f. 為 fo.
UPDATE_FORMA = '''
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, ' f. ', ' fo. ')
where criteria like  '品型%'
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_FORMA)
    conn.commit()

In [ ]:
# 更正一些奇怪的東西
UPDATE_OTHERS = '''
UPDATE nomenclature.twredlist2017
SET family = 'Typhaceae' WHERE family = 'Typhaceae ';
UPDATE nomenclature.twredlist2017
SET family = 'Typhaceae' WHERE family = 'Typhaceae ';
UPDATE nomenclature.twredlist2017
SET family = 'Tamaricaceae' WHERE family = 'Tamaricaeae';
UPDATE nomenclature.twredlist2017
SET family = 'Isoëtaceae' where family = 'Isoetaceae';
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, '#', '');
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, '\?', '');
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, '\*', '');
''' 
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_OTHERS)
    conn.commit()

## 新增作者不含學名之欄位

為了方便比對，將 fullname (含作者) 拆成學名(name)以及作者(authors)

In [5]:
QUERY = 'SELECT fullname FROM twredlist2017;'
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(QUERY)
        fullnames = curs.fetchall()
for i in range(len(fullnames)):
    fname = fullnames[i][0]
    nameFormatted = g.fmtname(fname, split=True, format_type='custom', 
                                 italic_b="", italic_e="")
    if "'" in fname:
        fname = fname.replace("'", "''")
    if "\"" in fname:
        fname = fname.replace("\"", "''")
    if "'" in nameFormatted[0]:
        nameFormatted[0] = nameFormatted[0].replace("'", "''")
    if "'" in nameFormatted[1]:
        nameFormatted[1] = nameFormatted[1].replace("'", "''")
    UPDATE = '''
        UPDATE twredlist2017 SET
            name = '%s', authors = '%s'
        WHERE fullname = '%s'
        ''' % (nameFormatted[0], nameFormatted[1], fname)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(UPDATE)
conn.commit()

### 更新屬名以及屬中名

屬中名的資料表 nomenclature.twp_genus

In [6]:
UPDATE_GENUS = '''
UPDATE nomenclature.twredlist2017
SET genus = split_part(name, ' ', 1);
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_GENUS)
conn.commit()

In [8]:
UPDATE_GENUS_ZH = '''
UPDATE nomenclature.twredlist2017
SET genus_zh = g.genus_zh
FROM (SELECT * FROM nomenclature.twp_genus) as g
WHERE twredlist2017.genus = g.genus
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_GENUS_ZH)
conn.commit()

In [ ]:
# 更新 plant_type
UPDATE_PLANT_TYPE = '''
UPDATE nomenclature.twredlist2017
SET plant_type = p.plant_type
FROM (SELECT * FROM nomenclature.plant_type) as p
WHERE twredlist2017.family = p.family;
UPDATE nomenclature.twredlist2017
SET plant_type = 3 WHERE plant_type = 4;
UPDATE nomenclature.twredlist2017
SET plant_type = 0 WHERE family='Lycopodiaceae' or
family = 'Isoëtaceae' or family = 'Selaginellaceae';
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_PLANT_TYPE)
conn.commit()

In [ ]:
# 未成對的 " 會造成 zhmakeindex 爛掉
UPDATE_BAMBUSA = '''
UPDATE nomenclature.twredlist2017
SET name = 'Bambusa dolichomerithalla cv. ''Green\x0astripe-stem\'\'',
authors = 'W. C. Lin'
WHERE zh_name = '金絲火廣竹';
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_BAMBUSA)
conn.commit()

In [ ]:
Q = '''SELECT name from twredlist2017 where name like '%×%';'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(Q)
        qry = curs.fetchall()

In [ ]:
FULLNAME = g.fmtnameNew('Cornopteris decurrenti-alata (Hook.) Nakai var. decurrenti-alata', 
             format_type= 'custom', italic_b='\\textit{', italic_e='}',
             withSpAuthor= True)[1]

In [ ]:
g.fmtnameNew('Cornopteris decurrenti-alata (Hook.) Nakai var. decurrenti-alata')

## 2017-12-19 最後資料彙整

因為和明後來有再修正，所以需要對照哪些地方有修正過

In [ ]:
#id|family_fot|family_fot_zh|family|family_zh|fullname|
#zh_name|prev_category|prev_criteria|prev_evaluator|
#category|criteria|evaluator|evaluation|images|endemic|native|
#others|note|update

CREATE_TABLE = '''
-- CREATE TABLE nomenclature.twredlist20171219_backup AS (SELECT * FROM nomenclature.twredlist2017);
DROP TABLE IF EXISTS nomenclature.twredlist20171219;                                                                                                                                   
CREATE TABLE nomenclature.twredlist20171219 (                                                                                                                                          
    id serial primary key,                                                                                                                                            
    plant_type integer,                                                                                                                                               
    family_fot varchar,                                                                                                                                               
    family_fot_zh varchar,                                                                                                                                            
    family varchar,                                                                                                                                                   
    family_zh varchar,                                                                                                                                                
    genus varchar,                                                                                                                                                    
    genus_zh varchar,                                                                                                                                                 
    name varchar,                                                                                                                                                     
    fullname varchar,                                                                                                                                                 
    zh_name varchar,                                                                                                                                                  
    authors varchar,                                                                                                                                                  
    prev_category varchar,                                                                                                                                            
    prev_criteria varchar,                                                                                                                                            
    prev_evaluator varchar,                                                                                                                                           
    category varchar,                                                                                                                                                 
    criteria varchar,                                                                                                                                                 
    evaluator varchar,                                                                                                                                                
    evaluation varchar,                                                                                                                                               
    images varchar,                                                                                                                                                   
    endemic varchar,                                                                                                                                                  
    native varchar,                                                                                                                                                   
    others varchar,                                                                                                                                                   
    note varchar,
    update varchar,
    tropicos_namecode varchar,
    tropicos_fullname varchar,
    ipni_id varchar,
    ipni_fullname varchar,
    ipni_name varchar
);
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(CREATE_TABLE)
    conn.commit()

In [ ]:
# 匯入 csv 檔
# 匯入前要把斷行檢查一下
# Limnocharis flava
# Sorghum bicolor subsp. arundinaceum

redlistRaw = []
with open('../../workspace/20171219/twredlist1219.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
r = redlistRaw

for row in range(len(r)):
    # escape quote (for postgresql)
    if "'" in r[row][5]:
        r[row][5] = r[row][5].replace("'", "''")
    eachRow = "'" + "','".join(r[row]) + "'"
    # replace full-width braces
    eachRow = eachRow.replace('（', ' (')
    eachRow = eachRow.replace(' ）', ') ')
    # replace 雜交的 "x" (alphabet) to × (cross sign)
    eachRow = re.sub(' x ', ' × ', eachRow)
    # replace spaces greater than 1 
    eachRow = re.sub(" {1,}",' ', eachRow)
    # replace trailing spaces 
    eachRow = re.sub(" {1,}'","'", eachRow)
    # replace non-break spaces
    eachRow = eachRow.replace('\xa0',' ')
    # replace ssp. to subsp.
    eachRow = re.sub(" ssp. ", " subsp. ", eachRow)
    
    INSERT_REDLIST = '''
    INSERT INTO nomenclature.twredlist20171219 
    (id,family_fot,family_fot_zh,family,family_zh,fullname,                                                                                                                   
    zh_name,category,criteria,endemic,                                                                                                                
    native) VALUES (%s)
    ''' % (eachRow)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(INSERT_REDLIST)
        conn.commit()

In [ ]:
# 匯入 csv 檔
# 匯入前要把斷行檢查一下
# Limnocharis flava
# Sorghum bicolor subsp. arundinaceum

redlistRaw = []
with open('../../workspace/20171219/newlyadded_1220.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
nA = redlistRaw

for row in range(len(nA)):
    # escape quote (for postgresql)
    if "'" in nA[row][5]:
        nA[row][5] = nA[row][5].replace("'", "''")
    eachRow = "'" + "','".join(nA[row]) + "'"
    # replace full-width braces
    eachRow = eachRow.replace('（', ' (')
    eachRow = eachRow.replace(' ）', ') ')
    # replace 雜交的 "x" (alphabet) to × (cross sign)
    eachRow = re.sub(' x ', ' ×', eachRow)
    # replace spaces greater than 1 
    eachRow = re.sub(" {1,}",' ', eachRow)
    # replace trailing spaces 
    eachRow = re.sub(" {1,}'","'", eachRow)
    # replace non-break spaces
    eachRow = eachRow.replace('\xa0',' ')
    # replace ssp. to subsp.
    eachRow = re.sub(" ssp. ", " subsp. ", eachRow)
    
    INSERT_REDLIST = '''
    INSERT INTO nomenclature.twredlist2017
    (id,family_fot,family_fot_zh,family,family_zh,fullname,                                                                                                                   
    zh_name,prev_category,prev_criteria,category,criteria,endemic,                                                                                                                
    native) VALUES (%s)
    ''' % (eachRow)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(INSERT_REDLIST)
        conn.commit()

In [ ]:
fullname = 'Oxalis acetosella L. subsp. griffithii (Edgew. & Hook f.) H.Hara var. formosana S.F.Huang & T.C.Huang'

In [ ]:
g.fmtnameNew(iname, format_type= 'custom', italic_b='\\textit{', italic_e='}')

# 1. 四名

In [ ]:
epithetLst = []                                                                                                                                
epithet = ''                                                                                                                                   
author_start = ''                                                                                                                              
cross_type = ''                                                                                                                                
subordinate_status = []
fullname_split = fullname.split(' ')
subrank = ['subsp.', 'ssp.', 'var.', 'fo.', 'cv.', '×', 'x']
for s in range(0,len(subrank)):                                                                                                                
      if subrank[s] in fullname_split:                                                                                                           
          subrank_idx = fullname_split.index(subrank[s])                                                                                         
          if subrank[s] == '×' or subrank[s] == 'x':                                                                                             
              # 把 x 取代成 ×                                                                                                                    
              subrank[s] = '×'                                                                                                                   
              fullname_split[subrank_idx] = '×'                                                                                                  
              # × 後面有跟著屬名的                                                                                                               
              # ex: Genus epithet × Genus epithet2                                                                                               
              if re.search('[A-Z].*', fullname_split[subrank_idx+1]):                                                                            
                  subordinate_status.append([subrank_idx, subrank[s]])                                                                           
                  cross_type = 1                                                                                                                 
              else:                                                                                                                              
                  # 沒有屬名的 case                                                                                                              
                  fullname_split[subrank_idx+1] = '×' + fullname_split[subrank_idx+1]                                                            
                  fullname_split.remove(subrank[s])                                                                                              
          else:                                                                                                                                  
              subordinate_status.append([subrank_idx, subrank[s]])                                                                               

          # check for autonyms                                                                                                                   
          epithet = fullname_split[1]                                                                                                            
          sub_epithet = fullname_split[subrank_idx + 1]                                                                                          
          if epithet == sub_epithet:                                                                                                             
              autonym = True         

## 處理 illegitimate & invalid 名稱

In [11]:
redlistRaw = []
with open('../../workspace/illegitimate.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
il = redlistRaw

In [12]:
QIL = ''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(CREATE_TABLE)

[['Helwingia japonica (Thunb.) Dietr. subsp. taiwaniana Y.P. Yang & H.Y. Liu',
  'Helwingia japonica subsp. formosana'],
 ['Geophila repens (L.) I.M. Johnst.', 'Geophila herbacea'],
 ['Epithema taiwanense S.S.Ying var. taiwanensis',
  'Epithema taiwanensis var. taiwanensis'],
 ['Alectra arvensis (Benth.) Merr.', 'Alectra avensis'],
 ['Swietenia mahagoni (L.) Jacq.', 'Swietenia mahogoni'],
 ['Epithema taiwanense S.S.Ying var. fasciculatum (C.B. Clarke) Z.Yu Li & M.T. Kao',
  'Epithema taiwanensis var. fasciculata'],
 ['-- Elaeagnus formosensis Hatus.', 'Elaeagnus ohashii'],
 ['Gastrodia flabilabella S.S.Ying', 'Gastrodia flavilabella'],
 ['Ficus ampelos Burm. f.', 'Ficus ampelas'],
 ['Phaenosperma globosum Munro ex Benth.', 'Phaenosperma globosa'],
 ['Diospyros blancoi A. DC.', 'Diospyros philippensis'],
 ['Codoriocalyx motorius (Houtt.) H. Ohashi', 'Codariocalyx motorius'],
 ['Malva cathayensis M.G. Gilbert, Y. Tang & Dorr', 'Malva sinensis'],
 ['Triumfetta rhomboidea Jacq.', 'Triumfet